Using Tensor Flow and SciKit Learn to create a Neural Network in an attempt to create a predictive model:

In [6]:
import tensorflow as tf
import pandas as pd
from sqlalchemy import create_engine
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

# Load the diabetes dataset
engine = create_engine('sqlite:///diabetesData.db')
diabetes_df = pd.read_sql_query('select * from diabetes', con=engine)

# Separate features (X) and target variable (y)
X = diabetes_df.drop("diabetes", axis=1)
y = diabetes_df["diabetes"].to_numpy()

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
input_dim = len(X_train_scaled[0])
node_value1 = 20
node_value2 = 10
node_value3 = 1
nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=node_value1, activation="relu", input_dim=input_dim))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=node_value2, activation="relu",))

# Output layer
nn.add(tf.keras.layers.Dense(units=node_value3, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 20)                360       
                                                                 
 dense_4 (Dense)             (None, 10)                210       
                                                                 
 dense_5 (Dense)             (None, 1)                 11        
                                                                 
Total params: 581
Trainable params: 581
Non-trainable params: 0
_________________________________________________________________


In [3]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [4]:
# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs=100)

Epoch 1/100
1657/1657 [==============================] - 3s 1ms/step - loss: 0.5237 - accuracy: 0.7376
Epoch 2/100
1657/1657 [==============================] - 2s 1ms/step - loss: 0.5075 - accuracy: 0.7499
Epoch 3/100
1657/1657 [==============================] - 2s 1ms/step - loss: 0.5053 - accuracy: 0.7516
Epoch 4/100
1657/1657 [==============================] - 2s 1ms/step - loss: 0.5041 - accuracy: 0.7513
Epoch 5/100
1657/1657 [==============================] - 2s 1ms/step - loss: 0.5035 - accuracy: 0.7522
Epoch 6/100
1657/1657 [==============================] - 2s 1ms/step - loss: 0.5029 - accuracy: 0.7528
Epoch 7/100
1657/1657 [==============================] - 2s 1ms/step - loss: 0.5023 - accuracy: 0.7531
Epoch 8/100
1657/1657 [==============================] - 2s 1ms/step - loss: 0.5021 - accuracy: 0.7546
Epoch 9/100
1657/1657 [==============================] - 2s 1ms/step - loss: 0.5017 - accuracy: 0.7549
Epoch 10/100
1657/1657 [==============================] - 2s 1ms/step - l

In [5]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

553/553 - 1s - loss: 0.5076 - accuracy: 0.7476 - 636ms/epoch - 1ms/step
Loss: 0.5076055526733398, Accuracy: 0.7476376295089722


Using Keras Tuner in an attempt to find the best predictive model:

In [9]:
import keras_tuner as kt

# Define the model-building function
def build_model(hp):
    model = tf.keras.models.Sequential()

    # First hidden layer: tuning the number of units
    model.add(tf.keras.layers.Dense(
        units=hp.Int('units_1', min_value=32, max_value=128, step=32),
        activation='relu',
        input_dim=X_train_scaled.shape[1]
    ))

    # Second hidden layer: tuning the number of units
    model.add(tf.keras.layers.Dense(
        units=hp.Int('units_2', min_value=16, max_value=64, step=16),
        activation='relu'
    ))

    # Output layer
    model.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

    # Compile the model: tuning the learning rate of Adam
    model.compile(
        optimizer=tf.keras.optimizers.Adam(
            learning_rate=hp.Float('learning_rate', min_value=1e-4, max_value=1e-2, sampling='LOG')
        ),
        loss='binary_crossentropy',
        metrics=['accuracy']
    )

    return model

# Initialize the tuner
tuner = kt.RandomSearch(
    build_model,
    objective='accuracy',
    max_trials=10,
    executions_per_trial=3,
    directory='keras_tuner_logs',
    project_name='keras_tuner_nn'
)

# Search for the best hyperparameters
tuner.search(X_train_scaled, y_train, epochs=10, validation_data=(X_test_scaled, y_test))

# Get the best hyperparameters
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
print(f"Best hyperparameters: {best_hps.values}")

# Build and train the model with the best hyperparameters
best_model = tuner.hypermodel.build(best_hps)
history = best_model.fit(X_train_scaled, y_train, epochs=20, validation_data=(X_test_scaled, y_test))

Trial 10 Complete [00h 01m 29s]
accuracy: 0.7544716795285543

Best accuracy So Far: 0.7568418780962626
Total elapsed time: 00h 15m 09s
Best hyperparameters: {'units_1': 128, 'units_2': 16, 'learning_rate': 0.001563799353197703}
Epoch 1/20
1657/1657 [==============================] - 3s 2ms/step - loss: 0.5132 - accuracy: 0.7469 - val_loss: 0.5051 - val_accuracy: 0.7527
Epoch 2/20
1657/1657 [==============================] - 2s 1ms/step - loss: 0.5052 - accuracy: 0.7522 - val_loss: 0.5059 - val_accuracy: 0.7499
Epoch 3/20
1657/1657 [==============================] - 2s 1ms/step - loss: 0.5042 - accuracy: 0.7534 - val_loss: 0.5049 - val_accuracy: 0.7523
Epoch 4/20
1657/1657 [==============================] - 3s 2ms/step - loss: 0.5027 - accuracy: 0.7528 - val_loss: 0.5041 - val_accuracy: 0.7513
Epoch 5/20
1657/1657 [==============================] - 2s 1ms/step - loss: 0.5018 - accuracy: 0.7542 - val_loss: 0.5046 - val_accuracy: 0.7519
Epoch 6/20
1657/1657 [==============================

In [12]:
model_loss, model_accuracy = best_model.evaluate(X_test_scaled, y_test, verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

553/553 - 0s - loss: 0.5093 - accuracy: 0.7500 - 465ms/epoch - 841us/step
Loss: 0.509268045425415, Accuracy: 0.7500141263008118
